# Google Review Keyword Frequency

## 수행 요소
1. keyword별 빈도수 리스트 파일로 저장 

## Data path 확인

In [2]:
!pwd # 현재 경로 확인 

/home/aiffel-dj19/jungcheck/EDA


아래 tree 명령어로 상위 폴더(jungcheck)내 파일 구성을 볼 수 있습니다.    

In [3]:
!ls . # 현재 위치의 file list

frequency  googleKeywordFrequency.ipynb


In [4]:
!tree -L 3 -N ./data/google_review # jungcheck 폴더내 경로 확인 

./data/google_review [error opening dir]

0 directories, 0 files


## Library Import

In [5]:
import os 
import glob
from tqdm import tqdm

import re
import pandas as pd 
import copy # deep copy

## Data Path & Load 

In [10]:
root = os.getenv('HOME') + '/jungcheck'
root

'/home/aiffel-dj19/jungcheck'

In [11]:
data_path = os.path.join(root, 'data/google_review/')
data_path

'/home/aiffel-dj19/jungcheck/data/google_review/'

In [12]:
folders = os.listdir(data_path)
print(folders)
print(len(folders)) # 17개 대상 관광지 중 15개 

['07장태산', '05계족산', '09동춘당', '11으능정이문화의거리', '02뿌리공원', '03엑스포', '13성심당', '14수통골', '12유성온천', '04한밭수목원', '17국립중앙과학관', '06대청호', '16대전문화예술단지', '13시민천문대', '01오월드']
15


In [13]:
keywords_ko = [file_name[2:] for file_name in folders]
keywords_en = ['jangtae_mountain', 'gyejok_mountain', 'dongchundang', 'uineungjeongi_street', 'ppuri_park', 
               'expo_science_park', 'sungsimdang_bakery', 'water_barrel', 'yuseong_hotspring', 'hanbat_arboretum', 
               'science_museum','daecheong_lake', 'art_culture_complex', 'observatory', 'oworld_zoo']

keywords = {k:v for k, v in zip(keywords_ko, keywords_en)}
keywords

{'장태산': 'jangtae_mountain',
 '계족산': 'gyejok_mountain',
 '동춘당': 'dongchundang',
 '으능정이문화의거리': 'uineungjeongi_street',
 '뿌리공원': 'ppuri_park',
 '엑스포': 'expo_science_park',
 '성심당': 'sungsimdang_bakery',
 '수통골': 'water_barrel',
 '유성온천': 'yuseong_hotspring',
 '한밭수목원': 'hanbat_arboretum',
 '국립중앙과학관': 'science_museum',
 '대청호': 'daecheong_lake',
 '대전문화예술단지': 'art_culture_complex',
 '시민천문대': 'observatory',
 '오월드': 'oworld_zoo'}

In [14]:
filtered_path = os.path.join(root, 'filtered/google_review/')
filtered_path

'/home/aiffel-dj19/jungcheck/filtered/google_review/'

In [15]:
folders = os.listdir(filtered_path)
print(folders)
print(len(folders)) # 17개 대상 관광지 중 15개 

['gyejok_mountain.csv', 'dongchundang.csv', 'observatory.csv', 'hanbat_arboretum.csv', 'expo_science_park.csv', 'uineungjeongi_street.csv', 'oworld_zoo.csv', 'daecheong_lake.csv', 'water_barrel.csv', 'ppuri_park.csv', 'jangtae_mountain.csv', 'science_museum.csv', 'yuseong_hotspring.csv', 'sungsimdang_bakery.csv', 'art_culture_complex.csv']
15


In [16]:
# key <=> value
keywords = {v:k for k, v in keywords.items()}
keywords

{'jangtae_mountain': '장태산',
 'gyejok_mountain': '계족산',
 'dongchundang': '동춘당',
 'uineungjeongi_street': '으능정이문화의거리',
 'ppuri_park': '뿌리공원',
 'expo_science_park': '엑스포',
 'sungsimdang_bakery': '성심당',
 'water_barrel': '수통골',
 'yuseong_hotspring': '유성온천',
 'hanbat_arboretum': '한밭수목원',
 'science_museum': '국립중앙과학관',
 'daecheong_lake': '대청호',
 'art_culture_complex': '대전문화예술단지',
 'observatory': '시민천문대',
 'oworld_zoo': '오월드'}

👉 영어 keyword로 한글로 된 keyword를 살펴볼거기 때문에 key와 value를 바꿔줍니다. 

In [17]:
# root_path내의 dataframe을 합해주는 함수
def concatCsv(root_path:str, files_list:list):
    data_list = []
        
    for file in files_list:
        file_path = os.path.join(root_path, file)
        print(file_path)

        df = pd.read_csv(file_path, encoding='utf-8')  # csv 파일 읽기
        data_list.append(df)

    df = pd.concat(data_list, axis=0)
    return df

In [18]:
filtered_df = concatCsv(filtered_path, folders)
len(filtered_df)

/home/aiffel-dj19/jungcheck/filtered/google_review/gyejok_mountain.csv
/home/aiffel-dj19/jungcheck/filtered/google_review/dongchundang.csv
/home/aiffel-dj19/jungcheck/filtered/google_review/observatory.csv
/home/aiffel-dj19/jungcheck/filtered/google_review/hanbat_arboretum.csv
/home/aiffel-dj19/jungcheck/filtered/google_review/expo_science_park.csv
/home/aiffel-dj19/jungcheck/filtered/google_review/uineungjeongi_street.csv
/home/aiffel-dj19/jungcheck/filtered/google_review/oworld_zoo.csv
/home/aiffel-dj19/jungcheck/filtered/google_review/daecheong_lake.csv
/home/aiffel-dj19/jungcheck/filtered/google_review/water_barrel.csv
/home/aiffel-dj19/jungcheck/filtered/google_review/ppuri_park.csv
/home/aiffel-dj19/jungcheck/filtered/google_review/jangtae_mountain.csv
/home/aiffel-dj19/jungcheck/filtered/google_review/science_museum.csv
/home/aiffel-dj19/jungcheck/filtered/google_review/yuseong_hotspring.csv
/home/aiffel-dj19/jungcheck/filtered/google_review/sungsimdang_bakery.csv
/home/aiffel-d

8417

In [19]:
for key_en, key_ko in keywords.items():
    cnt = filtered_df[filtered_df['keyword']==key_en]['search'].count()
    print(f'{key_en}과 연관된 검색어는 {cnt}개')
    print(list(set(filtered_df[filtered_df['keyword']==key_en]['search'])))
    print('-'*40)

jangtae_mountain과 연관된 검색어는 518개
['장태산자연휴양림숲속의집', '장태산자연휴양림메타세콰이어산림욕장', '장태산자연휴양림전망대', '장태산자연휴양림']
----------------------------------------
gyejok_mountain과 연관된 검색어는 483개
['계족산성', '계족산(봉우리읍내동)', '계족산황톳길']
----------------------------------------
dongchundang과 연관된 검색어는 328개
['동춘당공원', '회덕동춘당']
----------------------------------------
uineungjeongi_street과 연관된 검색어는 333개
['으능정이거리', '대전스카이로드', '으능정이문화거리']
----------------------------------------
ppuri_park과 연관된 검색어는 426개
['뿌리공원']
----------------------------------------
expo_science_park과 연관된 검색어는 782개
['세계엑스포기념품박물관', '엑스포과학공원', '한빛탑', '엑스포음악분수', '엑스포다리']
----------------------------------------
sungsimdang_bakery과 연관된 검색어는 1862개
['성심당본점', '성심당대전역점', '성심당케잌부띠끄본점', '성심당롯데백화점대전점', '성심당DCC점']
----------------------------------------
water_barrel과 연관된 검색어는 685개
['수통골유원지', '계룡산국립공원수통골지구']
----------------------------------------
yuseong_hotspring과 연관된 검색어는 432개
['유성온천족욕체험장', '유성온천공원']
----------------------------------------
hanbat_arboretum과 연관된

## Frequency

In [20]:
from soynlp.noun import LRNounExtractor
from soynlp.noun import LRNounExtractor_v2
from soynlp.noun import NewsNounExtractor

def get_noun_sorted_by_frequency(noun_extractor):
    """
    Description
    이미 학습되어 있는 noun_extractor의 noun을 
    frequency 내림차순으로 정렬하여 리턴함.
    """
    return_lst = list()
    nouns = noun_extractor.extract()
    for (each_noun, each_noun_info) in nouns.items():
        each_noun_dict = {
            "noun": each_noun,
            "frequency": each_noun_info.frequency, 
            "score": each_noun_info.score
        }
        return_lst.append(each_noun_dict)
    return_lst = sorted(return_lst, key=lambda x: x['frequency'], reverse=True)
    return return_lst

In [21]:
noun_extractor_dict = {
    "LRNounExtractor": LRNounExtractor(verbose=False), 
    "LRNounExtractor_v2": LRNounExtractor_v2(verbose=False), 
    "NewsNounExtractor": NewsNounExtractor(verbose=False)
}

/home/aiffel-dj19/anaconda3/envs/aiffel/lib/python3.7/site-packages/soynlp
local variable 'f' referenced before assignment
local variable 'f' referenced before assignment


### Sample

In [22]:
target = 'oworld_zoo'

In [23]:
test_list = filtered_df[filtered_df['keyword']==target]['comment']
# filtered_list = ' '.join(test_list)

In [24]:
for noun_ext_name, noun_ext in noun_extractor_dict.items():
    # train
    noun_ext.train(test_list)
    print(f"== extractor: {noun_ext_name}")
    # noun을 빈도 내림차순으로 정렬하여 리턴
    noun_lst = get_noun_sorted_by_frequency(noun_ext)
    for noun in noun_lst[:20]:
        print(noun)
    print("--" * 50)

== extractor: LRNounExtractor
{'noun': '좋아', 'frequency': 97, 'score': 0.9816450000000001}
{'noun': '동물', 'frequency': 86, 'score': 0.6618587111111112}
{'noun': '아이들', 'frequency': 55, 'score': 0.9746742380952381}
{'noun': '놀이기구', 'frequency': 49, 'score': 0.5038004444444445}
{'noun': '동물들', 'frequency': 41, 'score': 0.7504501538461539}
{'noun': '대전', 'frequency': 30, 'score': 0.6999128333333334}
{'noun': '사람', 'frequency': 25, 'score': 0.8774649999999999}
{'noun': '아이', 'frequency': 23, 'score': 0.8065679166666667}
{'noun': '시간', 'frequency': 23, 'score': 0.8958123}
{'noun': '오월드', 'frequency': 22, 'score': 0.8750070000000001}
{'noun': '코로나', 'frequency': 20, 'score': 0.99883}
{'noun': '애들', 'frequency': 17, 'score': 0.9988382727272728}
{'noun': '곳이', 'frequency': 17, 'score': 0.99972}
{'noun': '장소', 'frequency': 14, 'score': 0.9993850000000001}
{'noun': '놀이', 'frequency': 14, 'score': 0.9889071666666668}
{'noun': '버드랜드', 'frequency': 14, 'score': 0.9373406249999999}
{'noun': '최고', 'f

👉 NewsNounExtractor가 명사 중심으로 추출을 잘해내는 것을 확인했으므로 해당 명사추출기를 사용합니다.

In [25]:
for noun_dict in noun_lst:
    target_freq = 0
    if target in noun_dict['noun']:
        target_freq = noun_dict.get('frequency')
        continue
    print(noun_dict.get('noun'))
print('-'*20)
print(f'검색 관광지명인 {target}가 {target_freq}번 언급됐습니다.')

동물
아이
놀이
아이들
놀이기구
동물들
대전
사람
시간
가족
재미
코로나
애들
입장
놀기
장소
버드랜드
아기
놀이동산
최고
나들이
거리
날씨
함께
생각
오래
어린이
관리
플라워랜드
입장료
정도
주말
추억
자유
하루
가격
볼거리
마음
회권
오랜만
에버랜드
규모
유모차
사람들
식물
튤립
아동
개장
어린이들
랜드
입장권
유아
느낌
아름
아들
기대
가족들
어른들
호랑
실내
벚꽃
코스
평일이라
자유이용권
최적
대기시간
동반
기분
이야기
움직
모습
직들
저학년
겨울
단점
--------------------
검색 관광지명인 oworld_zoo가 0번 언급됐습니다.


In [26]:
noun_ext = noun_extractor_dict["NewsNounExtractor"]
noun_lst = get_noun_sorted_by_frequency(noun_ext)

data_list = []
for noun_dict in noun_lst:
    noun = noun_dict.get('noun')
    freq = noun_dict.get('frequency')
    score = noun_dict.get('score')
    data_list.append([target, noun, freq, score])

before postprocessing 461
_noun_scores_ 126
after postprocessing 75


In [27]:
noun_dict.keys()

dict_keys(['noun', 'frequency', 'score'])

In [28]:
pd.DataFrame(data=data_list, columns=['target']+list(noun_dict.keys()))

,target,noun,frequency,score
0,oworld_zoo,동물,134,0.661859
1,oworld_zoo,아이,83,0.888833
2,oworld_zoo,놀이,79,0.631162
3,oworld_zoo,아이들,59,0.943187
4,oworld_zoo,놀이기구,50,0.503800
...,...,...,...,...
70,oworld_zoo,모습,3,0.834607
71,oworld_zoo,직들,3,0.999185
72,oworld_zoo,저학년,3,0.999805
73,oworld_zoo,겨울,3,1.000000


### 전체 Set에 적용 후 저장

In [29]:
keywords

{'jangtae_mountain': '장태산',
 'gyejok_mountain': '계족산',
 'dongchundang': '동춘당',
 'uineungjeongi_street': '으능정이문화의거리',
 'ppuri_park': '뿌리공원',
 'expo_science_park': '엑스포',
 'sungsimdang_bakery': '성심당',
 'water_barrel': '수통골',
 'yuseong_hotspring': '유성온천',
 'hanbat_arboretum': '한밭수목원',
 'science_museum': '국립중앙과학관',
 'daecheong_lake': '대청호',
 'art_culture_complex': '대전문화예술단지',
 'observatory': '시민천문대',
 'oworld_zoo': '오월드'}

In [35]:
save_path = os.path.join(os.getcwd(), 'frequency/')
save_path

'/home/aiffel-dj19/jungcheck/EDA/frequency/'

In [36]:
noun_ext = noun_extractor_dict["NewsNounExtractor"]

for key_en, _ in keywords.items():
    noun_ext.train(filtered_df[filtered_df['keyword']==key_en]['comment'])
    noun_lst = get_noun_sorted_by_frequency(noun_ext)
    data_list = []
    for noun_dict in noun_lst:
        noun = noun_dict.get('noun')
        freq = noun_dict.get('frequency')
        score = noun_dict.get('score')
        data_list.append([key_en, noun, freq, score])
        df = pd.DataFrame(data=data_list, columns=['keyword']+list(noun_dict.keys()))
        df.to_csv(save_path + key_en + '.csv', header=True, index=False)

before postprocessing 460
_noun_scores_ 145
after postprocessing 83
before postprocessing 464
_noun_scores_ 154
after postprocessing 97
before postprocessing 220
_noun_scores_ 89
after postprocessing 58
before postprocessing 247
_noun_scores_ 86
after postprocessing 53
before postprocessing 374
_noun_scores_ 130
after postprocessing 81
before postprocessing 625
_noun_scores_ 189
after postprocessing 114
before postprocessing 1746
_noun_scores_ 463
after postprocessing 255
before postprocessing 537
_noun_scores_ 173
after postprocessing 111
before postprocessing 365
_noun_scores_ 110
after postprocessing 63
before postprocessing 367
_noun_scores_ 140
after postprocessing 83
before postprocessing 345
_noun_scores_ 102
after postprocessing 69
before postprocessing 356
_noun_scores_ 125
after postprocessing 78
before postprocessing 785
_noun_scores_ 251
after postprocessing 152
before postprocessing 17
_noun_scores_ 8
after postprocessing 5
before postprocessing 461
_noun_scores_ 126
after

👉 명사 추출기를 썼음에도 좋아요와 같은 동사가 남는 현상이 있었습니다.    
먼저, Mecab의 품사태깅을 활용해 한번더 명사를 정제해줬습니다. 

### 데이터 다시 정제 후 품사태깅

In [51]:
test = filtered_df['comment'].iloc[0]
test

'가버운 등 산하기 좋은 곳 황톳길 느낌 굿'

In [41]:
from konlpy.tag import Kkma, Hannanum, Komoran, Mecab, Twitter

mecab = Mecab()

## 전체 품사 태깅
speech_pos = mecab.pos(filtered_df['comment'].iloc[0])
print(speech_pos)

[('가버', 'NNP'), ('운', 'NNG'), ('등', 'NNB'), ('산', 'NNG'), ('하', 'XSV'), ('기', 'ETN'), ('좋', 'VA'), ('은', 'ETM'), ('곳', 'NNG'), ('황톳길', 'NNG'), ('느낌', 'MAG'), ('굿', 'NNG')]


👉 Mecab으로 하더라도 문장의 띄어쓰기 문제가 해결되지 않은 경우에는 품사태깅이 잘 이뤄지지 않음을 알 수 있습니다. 데이터 정제 방법을 고민했을 때 구글 검색에서 검색어를 정제해주는 기능을 활용하기 위해 셀레니움을 통해 읽어오는 방향을 시도해볼까 고민중.. 

![image](https://user-images.githubusercontent.com/69677950/120032454-2a469300-c035-11eb-99f2-4677f7d365e7.png)